# Turing Machines

[On Computable Numbers, with an Application to the Entscheidungsproblem](https://people.math.ethz.ch/~halorenz/4students/Literatur/TuringFullText.pdf)  
Proceedings of the London Mathematical Society Series 2, 42, 230-265  
Turing, A. M. (1937)  

## Long Multiplication

Multiply 123 by 45.  
You need your times tables for 1 to 9.  
They are a list of instructions.

```
 123
x 45
----
 615
4920
----
5535
```

```
123x45=
```

```
123x45=615,4920=5535
```

## Turing Machine Simulator

In [ ]:
# Input alphabet.
input_symbols = {'0', '1'}

# The Turing machine tape - infinite in both directions.
tape = '___________________________'

# The position on the tape that the machine is over.
pos = 0

# Tape alphabet.
tape_symbols = input_symbols.union({'_'})

# Allowed states.
states = {'A', 'B', 'C', 'D', 'ACCEPT', 'REJECT'}

# The initial state of the machine.
initial_state = 'A'

# The final state of the machine.
final_states = {'ACCEPT', 'REJECT'}

In [ ]:
state_table = [
    ['A', '0',           'B',      '0', 'R'],
    ['A', '1',           'B',      '1', 'R'],
    ['A', '_',           'ACCEPT', '_', 'R'],
    ['B', '0',           'C',      '1', 'R'],
    ['B', '1',           'D',      '0', 'R'],
    ['B', '_',           'REJECT', '_', 'R'],
    ['C', '0',           'C',      '0', 'R'],
    ['C', '1',           'C',      '1', 'R'],
    ['C', '_',           'A',      '_', 'L'],
    ['D', '0',           'D',      '0', 'R'],
    ['D', '1',           'D',      '1', 'R'],
    ['D', '_',           'A',      '_', 'L']
]

## Parity Check

In [ ]:
# States.
states = {'U', 'V', 'A', 'R'}

# Initial state.
initial_state = 'U'

# Final states.
final_states = {'A', 'R'}

# Position on the tape.
pos = 0

# Initial tape.
tape = '_'

# State table for an even parity checker.
state_table = {
    ('U', '0'): ('U', '0', 'R'),
    ('U', '1'): ('V', '1', 'R'),
    ('U', '_'): ('A', '_', 'L'),
    ('V', '0'): ('V', '0', 'R'),
    ('V', '1'): ('U', '1', 'R'),
    ('V', '_'): ('F', '_', 'L'),
}

def L():
    """Move the machine head left."""
    global pos, tape
    pos -= 1
    if pos < 0:
        pos = 0
        tape = ['_'] + tape

def R():
    """Move the machine head right."""
    global pos, tape
    pos += 1
    if pos >= len(tape):
        tape = tape + ['_']


def run():
    global tape, pos
    tape = list(tape)
    state = initial_state
    while state not in final_states:
        symbol = tape[pos]
        new_state, new_symbol, move = state_table[(state, symbol)]
        tape[pos] = new_symbol
        if move == 'L':
            L()
        elif move == 'R':
            R()
        state = new_state
    return state


In [20]:
run()

'A'

## End